In [ ]:
import pandas as pd
# CSV 파일에서 titleId 값을 가져오기
title_df = pd.read_csv("naver_listly_updated.csv")  # title.csv 파일을 불러오기
id_list = title_df['ID'].astype(str).tolist()  # titleId 칼럼의 값들을 문자열로 변환하여 리스트로 저장
print(len(id_list))
print(id_list[:10])

In [ ]:
# 전체 합친 코드

import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

# ID 리스트 (예: ['796152', '817346', '703846'] 등)
id_list = id_list[0:]  # 필요 시 직접 지정 가능

# URL 생성
base_url = 'https://comic.naver.com/webtoon/list?titleId='
urls = [f"{base_url}{num}" for num in id_list]

# WebDriver 설정
options = Options()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
driver = webdriver.Chrome(options=options)

# 데이터 저장 리스트
data = []

# URL 순회하며 정보 수집
for url in urls:
    driver.get(url)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # 제목
    title_h2 = soup.find("h2", class_="EpisodeListInfo__title--mYLjC")
    title_text = title_h2.get_text() if title_h2 else "제목을 찾을 수 없음"

    # 줄거리(스토리)
    story_p = soup.find("p", class_="EpisodeListInfo__summary--Jd1WG")
    story_text = story_p.get_text(strip=True) if story_p else "스토리를 찾을 수 없음"

    # 태그
    tags_div = soup.find("div", class_="TagGroup__tag_group--uUJza")
    if tags_div:
        tags = [tag.get_text() for tag in tags_div.find_all("a", class_="TagGroup__tag--xu0OH")]
        tags_text = ', '.join(tags)
    else:
        tags_text = "태그 그룹을 찾을 수 없음"

    # 에피소드 수 정보
    episode_info = soup.find("em", class_="ContentMetaInfo__info_item--utGrf")
    episode_info_text = episode_info.get_text(strip=True) if episode_info else "에피소드 정보를 찾을 수 없음"

    # 조회수 (좋아요 수)
    view_count = soup.find("span", class_="EpisodeListUser__count--fNEWK")
    view_count_text = view_count.get_text(strip=True) if view_count else "조회수를 찾을 수 없음"

    # 첫 공개일
    date_tag = soup.find("span", class_="date")
    date_text = date_tag.get_text(strip=True) if date_tag else "날짜를 찾을 수 없음"

    # 저장
    data.append({
        'Title': title_text,
        'Story': story_text,
        'Webtoon Info': episode_info_text,
        'Like Count': view_count_text,
        '1st Date': date_text,
        '#': tags_text
    })

    print("현재작품 :", title_text)
    time.sleep(10)

# 드라이버 종료
driver.quit()

# DataFrame 생성 및 출력
df = pd.DataFrame(data)[['Title', 'Story', 'Webtoon Info', 'Like Count', '1st Date', '#']]
df

In [ ]:
df.to_csv('naver_crawlling_all.csv', index=False, encoding='utf-8-sig')